In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
mkdir grace_wavs

In [10]:
ls

drive/  grace_wavs/  sample_data/


In [11]:
cd drive/My Drive/amazing_grace/

/content/drive/.shortcut-targets-by-id/108Ujtp8i29zJ2_EUinbLFEbkDJZjf7jb/amazing_grace


In [22]:
pip install pydub

In [23]:
import os
import argparse
import pydub
from pydub import AudioSegment

In [26]:
pip install ffmpeg-python

In [27]:
pip install --upgrade AudioConverter


In [28]:
ls

 amazing_grace@
'Colab Notebooks'/
'Copy of Visual Analysis Guide for PVC 2020.gdoc'
'Getting started.pdf'
'Mona Lisa Visual Analysis Guide for PVC 2020.gdoc'
 Research/


In [29]:
pip install ffmpeg

  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6083 sha256=eb13f073dbe206fb07db6bfdafa39dd5e17dc90703ca0204648a03904facce07
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg


In [30]:
cd amazing_grace/

/content/drive/.shortcut-targets-by-id/108Ujtp8i29zJ2_EUinbLFEbkDJZjf7jb/amazing_grace


In [31]:
pip install torch


In [ ]:
cd amazing_grace

/content/drive/.shortcut-targets-by-id/108Ujtp8i29zJ2_EUinbLFEbkDJZjf7jb/amazing_grace


In [32]:
import sys
i = 0
for audio_file in os.listdir('./'):
  #print(audio_file)
  if i == 3:
    break
  wav_filename = "test"+os.path.splitext(os.path.basename(audio_file))[0] + ".wav"
  x = AudioSegment.from_file(audio_file).export(wav_filename, 
                                      format="wav")
  os.system("cp './%s' '/content/grace_wavs'"%(wav_filename,))
  #sys.path.insert(0,x)
  #print(x)
  #path = "/content/drive/My Drive/"
  #cp './x' path 
  #print(f"Creating {wav_filename}...")
  i += 1

In [36]:
cd ..

/content


In [37]:
ls

drive/  grace_wavs/  sample_data/


In [38]:
cd grace_wavs

/content/grace_wavs


In [39]:
ls

test100947520_14247345.wav  test100949529_13884896.wav
test100949529_13512922.wav


In [41]:
cd drive/My Drive/amazing_grace

/content/drive/.shortcut-targets-by-id/108Ujtp8i29zJ2_EUinbLFEbkDJZjf7jb/amazing_grace


In [42]:
i = 0
for audio_file in os.listdir('./'):
  #print(audio_file)
  if i == 10:
    break
  os.system("cp './%s' '/content/grace_wavs'"%(audio_file,))
  i += 1

In [49]:
cd grace_wavs/

/content/grace_wavs


In [50]:
ls

100947520_14247345.m4a  100992845_200619044.m4a  test100947520_14247345.wav
100949529_13512922.m4a  101016666_37475717.m4a   test100949529_13512922.wav
100949529_13884896.m4a  101016666_42558595.m4a   test100949529_13884896.wav
100950773_22975448.m4a  101035573_143446212.m4a
100985136_55416202.m4a  101046674_13547863.m4a


In [51]:
pip install librosa

In [52]:
import tensorflow as tf
import librosa


In [54]:
ls

100947520_14247345.m4a  100992845_200619044.m4a  test100947520_14247345.wav
100949529_13512922.m4a  101016666_37475717.m4a   test100949529_13512922.wav
100949529_13884896.m4a  101016666_42558595.m4a   test100949529_13884896.wav
100950773_22975448.m4a  101035573_143446212.m4a
100985136_55416202.m4a  101046674_13547863.m4a


In [55]:
audio,sr = librosa.load('100947520_14247345.m4a')

In [58]:
print(audio)
print(type(audio))

[ 6.1480432e-07 -6.3389643e-07  6.2022878e-07 ... -4.1226191e-03
 -6.4668315e-03 -8.3919698e-03]
<class 'numpy.ndarray'>


In [75]:
tens = tf.constant(audio)
print(tens)
print(tens.numpy())

tf.Tensor(
[ 6.1480432e-07 -6.3389643e-07  6.2022878e-07 ... -4.1226191e-03
 -6.4668315e-03 -8.3919698e-03], shape=(4205728,), dtype=float32)
[ 6.1480432e-07 -6.3389643e-07  6.2022878e-07 ... -4.1226191e-03
 -6.4668315e-03 -8.3919698e-03]


In [84]:
def parse_function(filename, label):
    #file_string = tf.read_file(filename)
    print(type(filename))
    print(type(filename))
    print(filename)
    #audio,sr = librosa.load(filename)
    #audio = tf.constant(audio)
    #This will convert to float values in [0, 1]
    #image = tf.image.convert_image_dtype(image, tf.float32)
    return [1,2], label

In [85]:
parse_function('100947520_14247345.m4a','grace')

<class 'str'>
<class 'str'>
100947520_14247345.m4a


([1, 2], 'grace')

In [86]:
files = ['100947520_14247345.m4a','100949529_13512922.m4a','100949529_13884896.m4a']
labels = ['grace']*len(files)

In [87]:
dataset = tf.data.Dataset.from_tensor_slices((files, labels))
dataset = dataset.shuffle(len(files))
dataset = dataset.map(parse_function, num_parallel_calls=4)

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
Tensor("args_0:0", shape=(), dtype=string)


In [ ]:
import tensorflow as tf
import librosa

raw_audio = tf.io.read_file('test100929465_13510526.wav')
audio, sr = tf.audio.decode_wav(raw_audio)

InvalidArgumentError: ignored

In [ ]:
pip install librosa

In [ ]:
import pandas as pd
import tensorflow as tf
import librosa

AUTOTUNE = tf.data.experimental.AUTOTUNE


def get_dataset(files,labels):
    file_path_ds = tf.data.Dataset.from_tensor_slices(files)
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((file_path_ds, label_ds))


def load_audio(file_path, label):
    # Load one second of audio at 44.1kHz sample-rate
    audio = tf.io.read_file(file_path)
    audio, sample_rate = librosa.load(file_path)
    return tf.constant(audio), label


def prepare_for_training(ds, shuffle_buffer_size, batch_size):
    # Randomly shuffle (file_path, label) dataset
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    # Load and decode audio from file paths
    ds = ds.map(load_audio, num_parallel_calls=AUTOTUNE)
    # Repeat dataset forever
    ds = ds.repeat()
    # Prepare batches
    ds = ds.batch(batch_size)
    # Prefetch
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds


def main():
    # Load meta.csv containing file-paths and labels as pd.DataFrame
    #df = pd.read_csv('meta.csv')
    files = ['test100916686_13565095.wav','test100924597_137689990.wav','test100927035_143049118.wav']
    labels = ['grace']*3
    ds = get_dataset(files,labels)
    train_ds = prepare_for_training(ds,len(files),1)

    batch_size = 1
    train_steps = len(files) / batch_size

    #model = tf.keras.models.load_model('model.h5')
    #model.fit(train_ds, epochs=10, steps_per_epoch=train_steps)


if __name__ == '__main__':
    main()

TypeError: ignored